# Détection d'instants
Quand on dispose d'un signal temporel on cherche souvent à en analyser une partie. Pour cela il faut détecter un instant particulier ou délimiter des bornes (deux instants). A chaque fois qu'un instant doit être choisi, les analystes développent un code spécifique qui a toutes les chance d'être difficile à maintenir car des éléments inatendus peuvent survenir avec l'arrivée de nouveaux signaux.
Hors dans la majorité des cas l'instant qui intéresse un expert est clairement visible sur l'un des signaux temporels univarié, éventuellement la conjonction de deux signaux, mais rarement plus. Par ailleurs, sur le signal en question cette instant est facilement identifiable : il est proche d'une bosse ou d'un creu et en fonction de l'échelle d'observation on doit facilement le détecter puisque c'est un élément essentiellement visuel.

Ce module utilise un Opset en entrée, construit une interface permettant l'identification graphique par un expert de l'instant choisi sur quelques signaux.
Ensuite un algorithme fabrique des indicateurs basés sur la détection de creux et de bosses et extrait des points au hasard sur les courbes visualisées par l'expert. Chacun de ces points est un vecteur de grande dimension dont les coordonnées correspondent à des indicateurs de position (numéro et position des creux et bosses) à différentes échelles.
L'algorithme essaye de détecter si les points tirés sont à gauche (-1) ou à droite (+1) de l'instant sélectionné.

Finalement un indicateur de probabilité de présence de l'instant est construit et une détection est faite avec plus ou moins de confiance.

L'expert peut tester son détecteur et éventuellement le corriger en ajoutant des identifications manuelles et en relançant l'apprentissage. 

In [1]:
from importlib import reload

import numpy as np
from scipy import signal
import pandas as pd
import tabata.instants as instants

Pour des raisons de debuggage, je recharge le module.

In [2]:
reload(instants)

<module 'tabata.instants' from 'H:\\wrk\\_jlpath\\tabata\\instants.py'>

## Création du sélecteur d'instants.
On utilise les données retraitées depuis le notebook _opset_doc_. Le Selector est un Opset diposant de fonctions de sélection dans son interface.

In [3]:
fieldname = 'data/out/AFL1EB_C.h5'
S = instants.Selector(fieldname)

On affiche les observations (comme pour Opset), mais la différence est dans l'utilisation du clic de souris. En cliquant sur la courbe on peut sélectionner un instant.

In [4]:
S.plot(colname="ALT")

Les points selectionnés sont stockés dans la variable `selected` et les signaux observés sont dans `viewed`. On a cette seconde variable car un expert qui voit un signal mais ne sélectionne pas d'instant donne une information qu'il est aussi possible d'exploiter.

In [5]:
print('Instants selectionnés :\n', S.selected)
print('Signaux observés :\n', S.viewed)

Instants selectionnés :
 {}
Signaux observés :
 {0}


Pour des raisons de test nous supposerons que les instants et observations ont été sauvegardées.

In [6]:
S.selected = \
{0: ('ALT[m]', 1960),
 1: ('ALT[m]', 1821),
 3: ('ALT[m]', 1732),
 4: ('ALT[m]', 1600),
 5: ('ALT[m]', 1391),
 6: ('ALT[m]', 1736),
 7: ('ALT[m]', 1615)}

S.viewed = {0, 1, 2, 3, 4, 5, 6, 7}

Pour les instants, la sauvegarde est un dictionnaire indexé par le numéro d'enregistrement (signal) avec pour valeur le couple formé de la variable visualisée (en dernier) et de l'instant détecté.

## Création des indicateurs.
Les indicateurs sont des compteurs de bosses et de creux. Une bosse ou un creu est détecté par le passage par zéro de la drivée du signal. On utilise des échelles différentes pour lisser le signal et on regardera en plus des bosses et des creux les points d'inflexion qui sont les bosses et creux de la dérivée.

L'algorithme de lissage employé est un filtre de Savitsky-Golay : un filtre polynômial que l'on utilise avec des polynômes de degré 2. On fixe une largeur de filtre et sur chaque fenêtre on essaye estime la courbe par une parabole, le résultat du filtre est la valeur au centre de la fenêtre. L'utilisation d'un polynôme de degré deux évite la surparamétrisation et autorise le calcul des dérivées d'ordre 1 et 2.

In [7]:
dsi = S.make_indicators()

La méthode `make indicator()` renvoie un nouveau Selector qui a ses propriétés `selected` et `viewed` vides, mais pas sa propriété `computed`. Cette dernière affiche les points sélectionnés précédemment par des traits pointillés en bleu. Ils ne sont pas modifiable contrairement aux sélection violettes.

In [8]:
dsi.plot()

En parcourant cet Opset, vous remarquerez que seules les opérations sélectionnées sont enregistrées. D'autre part, en regardant la liste des variables on voit des noms un peu originaux :

* LEN correspond à la position absolue dans le signal ;
* REV est la position absolue inverse ;
* PERCENT est la position relative par rapport à la longuer du signal.

Les unités _pts_ et _%_ représentent le numéro de point et le pourcentage.

Les autres variables sont toutes de nom correspondant à la variable originale sélectionnée et l'unité ce code ainsi : _w#o#u#_ ou _w#o#d#_.

* _w_ correspond à la demi-largeur du filtre ;
* _o_ est l'ordre de dérivation (1 ou 2) ;
* _u_ et _d_ représentent un passage par zéro en montant (up) ou en descendant (down).

Ces valeurs sont issues du paramétrage par défaut de la méthode :

     def make_indicators(self, 
                        filename=None,
                        range_width = None,
                        range_sigma = None,
                        max_order=2):
                        
Le nom du fichier par défaut sera le même que celui du Selector (même répertoire) avec le préfixe '_I'.

* `range_width` est un itérateur donnant les demi-largeurs qui correspond aux dix premier centièmes de la longueur du plus petit signal (10 étant la plus petite valeur acceptable). Pour un signal de 1000 points on obtient `range(10,101,10)`.
* `range_sigma` est un itérateur des seuil permettant de détecter les passages par zéro, ce sont des multiples de l'écart-type local du signal lissé. Si le seuil est positif or regarde les passage par zéro en montant et s'il est négatif, les passages par zéro en descendant. Par défaut on prend `range(5,26,10)` et son opposé soit -25 -15 -5 5 15 25.
* `max_order` est l'ordre maximal de dérivation. Le degré du polynome estimé est égal à 2 par défaut sauf si on recherche des indicateurs d'ordre plus élevé, auquel cas le degré du polynome sera aussi augmenté.